# SPECFEM Users Workshop (Oct. 5-7, 2022)
-------------------------

## Part 0: Container Testing Instructions

- Day 0 instructions are meant for Users to test out their Docker container to ensure that things will go smoothly during the workshop.  
- Estimated times are provided next to each task to provide a benchmark on how fast thing should run. 
- Benchmarks were run on a 2015 Macbook Pro (OSX 10.14.6) with 3.1GHz Intel Core i7, and 16Gb of ram. 
- If you run into any issues while running through this notebook, please copy the error message and make note of which cell created the error.
- These instructions should be run from inside the Docker container, using Jupyter Lab (see *Docker Preamble*). 

----------------

**Docker Preamble:**  

To open this container you will need the latest Docker Image:  
**> Please run the following from your computer's Terminal**

```bash
docker pull ghcr.io/seisscoped/adjtomo:ubuntu20.04
```

To save the results we obtain from inside our container, we will need to mount our local filesystem.  
**> Please `cd` (change directory) to am empty working directory before running the following command:**

```bash
docker run -p 8888:8888 --mount type=bind,source=$(pwd),target=/home/scoped/work --shm-size=1gb ghcr.io/seisscoped/adjtomo:ubuntu20.04 
```

>__Docker commands explanation__: `pull` downloads the Docker Image from GitHub. `run` launches the container. `--mount`  binds our local filesystem (in the current working directory, pwd) with the container's internal filesytem (at location /home/scoped/work). `--shm-size` tells Docker to give us 1 Gigabyte of shared memory, which is required for MPI processes.

>__M1 Macs__: Docker is not well optimized for Macs running M1 chips, as compared to those running Intel chips. This results in Docker running $\sim$2$-$4x slower on M1 Macs. If you have an M1 we suggest you find an alternative for the workshop (Intel Mac, Windows, Linux) but if that is not possible, you can optimize your container slightly by adding the flag `--platform linux/amd64` to the 'docker run' command above (e.g., `docker run -p 8888:8888 --platform linux/amd64 ...`). With that said, please be prepared for overall slower run times, we will try to accomodate this as much as possible. 

>__SIGBUS Error__: If you are receiving the error message *Program received signal SIGBUS: Access to an undefined portion of a memory object.*, please `File -> Shut Down` the current container and reopen with a larger value for the `--shm-size` flag above. e.g., `--shm-size=2gb`. 

--------------------

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.
- To time a task, put a `%time` before the command (e.g., `%time ! ls`)


**Relevant Links:** 
- Today's notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_0_container_testing.ipynb
- Today's completed notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_0_container_testing.ipynb
- Workshop announcement: https://sites.google.com/alaska.edu/carltape/home/research/specfem2022?authuser=0
- adjTomo Container: https://github.com/SeisSCOPED/adjtomo
- adjTomo Org Page: https://github.com/adjTomo
- SeisFlows docs: https://seisflows.readthedocs.io/en/devel/
- Pyatoa docs: https://pyatoa.readthedocs.io/en/devel/
- PySEP readme: https://github.com/uafgeotools/pysep#readme

--------------

In [ ]:
# 0) Python import for in-notebook visualizations
from IPython.display import Image

## 1) Testing SPECFEM2D

We want to make sure we can run SPECFEM2D natively by running the example problem using MPI.  
**Benchmark time for cell w/ 4 MPI processes: ~60s**

In [ ]:
%cd /home/scoped/specfem2d

# Convenience function to edit the SPECFEM Par_file from the command line
! seisflows sempar -P DATA/Par_file nproc 4  # change number of processors from 1 to 4

! mpirun -n 4 bin/xmeshfem2D
! mpirun -n 4 bin/xspecfem2D

Using the navigation menu on the left, navigate to:

`home -> specfem2d -> OUTPUT_FILES`

**Open** any of the `forward_image*.jpg` files and see that the wavefront makes sense. It should look like a layered halfspace with topography and 3 layers.
One such figure is shown below.

In [ ]:
Image("OUTPUT_FILES/forward_image000000200.jpg")

### Testing Record Section (RecSec) Tool with SPECMFE2D Synthetics
- Make sure we can plot record sections (using PySEP) from SPECFEM2D synthetic waveforms.
- Must point RecSec at 1) synthetics, 2) station metadata, 3) source metadata
- The `--cartesian` flag tells RecSec that the SPECFEM2D domain is cartesian (not geographic)

In [ ]:
# Try RecSec with SPECFEM2D synthetics
! recsec --syn_path OUTPUT_FILES/ --cmtsolution DATA/SOURCE --stations DATA/STATIONS --cartesian

Using the navigation menu on the left, navigate to:

`home -> work -> specfem2d`

**Open** the `record_section.png` file to look at the created record section.


In [ ]:
Image("record_section.png")

## 2) SeisFlows Setup

Some directory bookkeeping to make sure we keep the container's `/home` directory free of clutter

In [ ]:
%cd /home/scoped/work  

! mkdir example_1 example_2 example_2a example_3 

>__NOTE:__ Successfully run examples will end with the following log message 

```
================================================================================
EXAMPLE COMPLETED SUCCESFULLY
================================================================================
```

## 3) Example 1: Homogeneous Halfspace Inversion

- This example runs a 1 iteration inversion for a single source-receiver pair, using two homogeneous halfspace models.  
- See [SeisFlows Example \#1 docs page](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#example-1-homogenous-halfspace-inversion) for guidance on what is going on  
- **Benchmark time for cell w/ 2 MPI processes: ~1m45s**

In [ ]:
# Run Example 1 with 2 processors
%cd /home/scoped/work/example_1
! seisflows examples run 1 -r /home/scoped/specfem2d --with_mpi --nproc 2

In [ ]:
# Plots the initial and final models, as well as the gradient 
! seisflows plot2d MODEL_INIT vs --savefig m_init_vs.png
! seisflows plot2d MODEL_TRUE vs --savefig m_true_vs.png
! seisflows plot2d GRADIENT_01 vs_kernel --savefig g_01_vs.png
! seisflows plot2d MODEL_01 vs --savefig m_01_vs.png

Using the navigation menu on the left, navigate to:

`home -> work -> example_1`

and **open** each of the .png files that were created to look at the results of this simple inversion. They should match the figures shown in the documentation page.

In [ ]:
Image("m_init_vs.png")

In [ ]:
Image("m_true_vs.png")

In [ ]:
Image("g_01_vs.png")

In [ ]:
Image("m_01_vs.png")

## 4) Example 2a: Re-create a Kernel from Tape et al. 2007

- This Example runs Example 2, for 1 event and 32 stations, comparing a homogeneous halfspace model with a checkerboard model. It generate a gradient and updated model.  
- See [SeisFlows Example \#2a docs page](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#re-creating-kernels-from-tape-et-al-2007) for guidance on what is going on.  
- **NOTE**: You can choose which event ID you're running by changing the integer after `--event_id`. Tape et al. show results for Event IDs 1 through 7.
- **Benchmark time for cell: ~4m**

In [ ]:
# Run Example 2 for only 1 iteration, and for a given event ID
%cd /home/scoped/work/example_2a
! seisflows examples run 2 -r /home/scoped/specfem2d --with_mpi --niter 1 --event_id 7  # <-- Choose your event by changing this value

In [ ]:
# Plots the target and final models, as well as the gradient 
! seisflows plot2d MODEL_TRUE vs --savefig m_true_vs.png
! seisflows plot2d MODEL_01 vs --savefig m_01_vs.png
! seisflows plot2d GRADIENT_01 vs_kernel --savefig g_01_vs.png

Using the navigation menu on the left, navigate to:

`home -> work -> example_2a`

and **open** the .png file that was created. Make sure that the kernel looks like one of the panels provided in [Figure 9 of Tape et al.](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#re-creating-kernels-from-tape-et-al-2007)

In [ ]:
Image("m_true_vs.png")

In [ ]:
Image("g_01_vs.png")

In [ ]:
Image("m_01_vs.png")

### Waveform Figures

We can also look at waveform figures generated by `Pyatoa`, a misfit quantification package used by SeisFlows to determine data-synthetic misfit. These can be found be navigating to

`home -> work -> example_2 -> output -> pyaflowa -> figures`

There is one .pdf file in there, corresponding to waveforms generated during the first iteration (i01) and first step count (s01). If you open the PDF you'll see waveform comparison figures showing time shifts between data and synthetics.


### Re-running Examples

If you want to **re-run this example** to generate a new event kernel, you will have to remove the SeisFlows state file. 
This bypasses SeisFlows' checkpointing feature, which will not allow you to run another workflow within an existing workflow directory.

In [ ]:
! rm /home/scoped/work/example_2a/sfstate.txt
! seisflows examples run 2 -r /home/scoped/specfem2d --with_mpi --niter 1 --event_id 5

## 5) Example 3: En-masse Forward Simulations

- This Example runs forward simulations for 250 source-receiver pairs (10 events, 25 stations). 
- See [SeisFlows Example \#3 docs page](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#example-3-en-masse-forward-simulations) for guidance on what is going on. 
- **Benchmark time for cell: ~4m**

In [ ]:
# Run Example 3
%cd /home/scoped/work/example_3
! seisflows examples run 3 -r /home/scoped/specfem2d --with_mpi --nproc 4

In [ ]:
# Plots one synthetic waveform exported by the Solver
! seisflows plotst output/solver/001/syn/AA.S000000.BXY.semd --savefig AA.S000000.BXY.semd.png

# Plots multiple synthetic waveforms exported by the Solver
! seisflows plotst output/solver/001/syn/AA.S00000?.BXY.semd --savefig AA.S00000n.BXY.semd.png

Using the navigation menu on the left, navigate to:

`home -> work -> example_3`

and **open** the .png files that were created to look at the waveforms. The waveforms should be relatively simple since this is a homogeneous halfspace model.

In [ ]:
Image("AA.S000000.BXY.semd.png")

In [ ]:
Image("AA.S000000.BXY.semd.png")

## 6) Testing SPECFEM3D

- We want to make sure we can run SPECFEM3D natively by running the homogeneous halfspace example problem using MPI.
- The cell first sets up the example directory, and then runs a 3D example problem with 4 processors.  
- We can not easily visualize SPECFEM3D results, so we plot synthetic waveforms as a check.
- **Benchmark time for cell: ~9m**

In [ ]:
%cd /home/scoped/specfem3d/EXAMPLES/homogeneous_halfspace

# Setup example problem
! ln -s /home/scoped/specfem3d/bin .
! cp -r ./meshfem3D_files ./DATA/meshfem3D_files
! mkdir -p OUTPUT_FILES/DATABASES_MPI

# Run SPECFEM3D exectuables
! mpirun -n 4 bin/xmeshfem3D
! mpirun -n 4 bin/xgenerate_databases
! mpirun -n 4 bin/xspecfem3D

In [ ]:
# Plots synthetic waveforms as a check
! seisflows plotst OUTPUT_FILES/*semd --savefig specfem3d_synthetics.png

Using the navigation menu on the left, navigate to:

`home -> specfem3d -> EXAMPLES -> homogeneous_halfspace`

and **open** the `specfem3d_synthetics.png` file to look at the waveforms. We can see the directivity of the moment tensor in the variation of amplitude of the given waveforms.

In [ ]:
Image("specfem3d_synthetics.png")

```python
All done. Thanks :)
```